In [1]:
%load_ext autoreload
%autoreload 2

## Calculate time horizon for loss of potential predictability

In [2]:
import xarray as xr
import numpy as np
import gfdl_utils as gu
import glob
from dask.diagnostics import ProgressBar
from information import *
from processing import *
from variance import *
import os

In [38]:
ppname = 'ocean_month'
frequency = gu.core.get_timefrequency(ppcontrol,ppname)
startmonth = 10
filename = '.'.join([ppname,str(startmonth).zfill(2),'zarr'])
ppp = xr.open_zarr(ppeDict['datasavedir']+'/processed/ppp/'+filename)

In [39]:
ppp

<xarray.Dataset>
Dimensions:  (nv: 2, time: 36, yh: 576, xh: 720, xq: 720, yq: 576, zl: 75)
Coordinates:
  * nv       (nv) float64 1.0 2.0
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 26 27 28 29 30 31 32 33 34 35
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * xq       (xq) float64 -299.5 -299.0 -298.5 -298.0 ... 58.5 59.0 59.5 60.0
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * yq       (yq) float64 -77.82 -77.63 -77.45 -77.26 ... 89.37 89.58 89.79 90.0
  * zl       (zl) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
Data variables:
    tos      (time, yh, xh) float32 dask.array<chunksize=(12, 576, 720), meta=np.ndarray>

In [40]:
def calc_timehorizon(da,threshold,dim='time',smooth=None,):
    ''' Find time at which value of [da] drops below [threshold], moving forward in time.
    If it never drops below threshold, np.max(time)+1 is returned.'''
    if smooth is not None:
        tmp = da.rolling({dim:smooth},center=True,min_periods=1).mean()
    else:
        tmp = da.copy()
    
    return xr.where(tmp<threshold, tmp[dim],np.max(da[dim])+1).min(dim=dim,skipna=True)

In [42]:
threshold = 0.235559205
smooth = 12

savedir = ppeDict['datasavedir']+'/processed/ppp_timehorizon/'

for variable,da in ppp.items():
    savenames = [variable,frequency,str(startmonth).zfill(2)]
    if smooth is not None:
        savenames.append('smooth_'+str(smooth).zfill(2))
    savefilename = '.'.join([*savenames,'nc'])

    tmp = calc_timehorizon(da,threshold,dim='time',smooth=smooth)
    tmp.name = variable
    print('Saving '+savefilename)
    with ProgressBar():
        tmp.to_netcdf(savedir+savefilename,mode='w')

Saving tos.monthly.10.smooth_12.nc
[########################################] | 100% Completed | 613.92 ms
